<a href="https://colab.research.google.com/github/Samhithgb/PyTorchBrushUp/blob/master/WeatherPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline  

In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate 
from zipfile import ZipFile
import requests 

In [ ]:
bs = 64
  

In [ ]:
def download_urls(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

download_urls("https://data.mendeley.com/datasets/4drtyfjtfy/1/files/a03e6097-f7fb-4e1a-9c6a-8923c6a0d3e0/dataset2.zip?dl=1","drive/My Drive/mendel.zip")

In [ ]:
with ZipFile("drive/My Drive/mendel.zip", 'r') as zip: 
  zip.extractall("drive/My Drive/mendelextract")

In [ ]:
path = start_path = Path('drive/My Drive/mendelextract')
path.ls()
path_data = path/'dataset2'

path_data.ls()
fnames = get_image_files(path_data)
fnames[:5]

In [ ]:
doc(ImageDataBunch.from_name_func)

In [ ]:
np.random.seed(2)


def get_labels(file_path): 
  o = str(file_path)
  if 'shine' in o: 
    return "shine" 
  elif 'sunrise' in o: 
    return "sunrise"
  elif 'rain' in o: 
    return 'rain'
  elif 'cloudy' in o: 
    return "cloudy"

data = ImageDataBunch.from_name_func(path_data, fnames, label_func=get_labels, ds_tfms= get_transforms(), size=24)
data.classes

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
learner = cnn_learner(data,models.resnet34,metrics=error_rate)

In [ ]:
learner.model

In [ ]:
doc(learner.fit_one_cycle)

In [ ]:
learner.fit_one_cycle(3)

In [ ]:
learner.save('stage-1')

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=2)